In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.metrics import classification_report, confusion_matrix
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Conv2D, Flatten, MaxPooling2D   
from keras.optimizers import RMSprop
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from matplotlib.pyplot import imshow
import glob
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1916200988971139077
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13364566112130866944
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
number_of_files = 26
dfs = []

for file_num in range(0,number_of_files):
    url = "../0616018/data"+str(file_num)+"_char.csv"
    t = pd.read_csv(url, index_col=None, header=0)
    dfs.append(t)

df  = pd.concat(dfs, axis=0, ignore_index=True)
df=shuffle(df)

In [4]:
df.count()

id          123637
category    123637
words       123637
dtype: int64

In [5]:
y = df.category
X = df.words
words_processed = [ ]
for i in df.words:
    words_processed.append(i)
X = [str(i) for i in words_processed]
print(X)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
le=LabelEncoder()
le.fit(df['category'].drop_duplicates(keep='first'))

y_encoder=[]
y_encoder=le.transform(y)

X_encoder=[]
vectorizer = CountVectorizer(analyzer='char')
X_encoder = vectorizer.fit_transform(X)

In [9]:
test_ratio=0.3
test_size=int (test_ratio*df['category'].count())
X_train, X_test, y_train, y_test = X_encoder.toarray()[test_size:], X_encoder.toarray()[:test_size], y_encoder[test_size:], y_encoder[:test_size] 

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X_encoder.toarray(), y_encoder, test_size = 0.2, random_state=43)

In [10]:
y_train_onehot=keras.utils.to_categorical(y_train)
y_test_onehot=keras.utils.to_categorical(y_test)

In [11]:
print(X_train.shape)
print(X_test.shape)

(84723, 8240)
(36309, 8240)


In [23]:
from keras.layers import Dense, Activation
model = Sequential([
    Dense(500, input_shape=(8240,)),
    Activation('relu'),
     Dense(300),
    Activation('relu'),
     Dense(300),
    Activation('relu'),
    Dense(125),
    Activation('softmax'),
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 500)               4120500   
_________________________________________________________________
activation_21 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 300)               150300    
_________________________________________________________________
activation_22 (Activation)   (None, 300)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 300)               90300     
_________________________________________________________________
activation_23 (Activation)   (None, 300)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 125)               37625     
__________

In [24]:
#Compile part
'''Your code here'''
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy']) 
#fit part
'''Your code here'''
y_train_history = model.fit(x=X_train, y=y_train_onehot, validation_split=0.2, epochs=10, batch_size=428, verbose=1)  

Train on 67778 samples, validate on 16945 samples
Epoch 1/10
67778/67778 [==============================] - 42s 622us/step - loss: 4.9818 - acc: 0.0924 - val_loss: 3.7597 - val_acc: 0.2307
Epoch 2/10
67778/67778 [==============================] - 37s 539us/step - loss: 2.5532 - acc: 0.4404 - val_loss: 2.5453 - val_acc: 0.4404
Epoch 3/10
67778/67778 [==============================] - 36s 537us/step - loss: 1.6758 - acc: 0.6064 - val_loss: 1.7436 - val_acc: 0.6130
Epoch 4/10
67778/67778 [==============================] - 38s 554us/step - loss: 1.3161 - acc: 0.6825 - val_loss: 1.3929 - val_acc: 0.6610
Epoch 5/10
67778/67778 [==============================] - 37s 543us/step - loss: 1.0789 - acc: 0.7302 - val_loss: 1.3097 - val_acc: 0.6827
Epoch 6/10
67778/67778 [==============================] - 39s 574us/step - loss: 0.9284 - acc: 0.7643 - val_loss: 1.1199 - val_acc: 0.7285
Epoch 7/10
67778/67778 [==============================] - 38s 553us/step - loss: 0.8265 - acc: 0.7904 - val_loss: 1.

In [25]:
scores = model.evaluate(X_test, y_test_onehot)
print('acc:',scores[1])

36309/36309 [==============================] - 13s 369us/step
acc: 0.7494560577382255
